In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pykospacing import Spacing
from kiwipiepy import Kiwi
import re
from collections import Counter
import csv
import json

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
df = pd.read_csv("./data/shampoo/oliveyoung_cleaned1.csv")

# 드라이샴푸, 스케일러 제외

In [4]:
# 드라이샴푸 제품명
dry = []
for item in df["제품명"].unique():
    if "드라이" in item:
        dry.append(item)

In [5]:
# 스케일러 제품명
scaler = []
for item in df["제품명"].unique():
    if "스케일러" in item:
        scaler.append(item)

In [6]:
# 드라이샴푸와 스케일러 제거
newdf = df[(~df["제품명"].isin(dry)) & (~df["제품명"].isin(scaler))]

In [8]:
# 남은 제품 수
newdf["제품명"].nunique()

475

# 한국어가 한 글자도 안 들어간 리뷰 제거

In [23]:
df = pd.read_csv("./data/shampoo/oliveyoung_labeling.csv")

C:\Users\Woojung\AppData\Local\Temp\ipykernel_23452\3626596301.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data/shampoo/oliveyoung_labeling.csv")


In [4]:
df.head()

,제품명,브랜드,원가,판매가,리뷰수,평균별점,5점비율,4점비율,3점비율,2점비율,...,향/다소아쉬워요,작성자,작성일자,리뷰,별점,사용현황,세정력,자극도,거품,향
0,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,4%,용자입니다,2025.05.19,일주일째 쓰는중인데 좋은거 같기도하고 아닌것 같기도 하고 한달뒤 후기로..ㄹ,5점만점에 5점,NaN,아주 만족해요,자극없이 순해요,거품이 풍성해요,아주 만족해요
1,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,4%,레몽몽,2025.04.28,거품이나 향은 무난한데 새치커버력이 홍보대비 좀 아쉽네요,5점만점에 4점,NaN,보통이에요,자극없이 순해요,거품이 풍성해요,아주 만족해요
2,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,4%,k202401158****,2025.05.16,얼마전에 구매했는데 쿠폰이 있어서 또 구매했어요 미리 쟁여 둘려구요,5점만점에 5점,재구매,아주 만족해요,자극없이 순해요,거품이 풍성해요,아주 만족해요
3,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,4%,k202401158****,2025.05.10,염색 하지 않고 몇년째 계속 사용하고 있어요남편 덕분에 알게 되어서 지금까지 같이 쓰고 있어요~,5점만점에 5점,NaN,아주 만족해요,자극없이 순해요,거품이 풍성해요,아주 만족해요
4,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,4%,sweetvio****,2025.05.06,남편40대되고 흰머리가많아 이걸사줬는데그레이정도는되는것같아요,5점만점에 5점,한달이상사용,보통이에요,보통이에요,보통이에요,보통이에요


In [5]:
df.shape

(182233, 32)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182233 entries, 0 to 182232
Data columns (total 32 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   제품명          182233 non-null  object 
 1   브랜드          182233 non-null  object 
 2   원가           121684 non-null  object 
 3   판매가          182233 non-null  object 
 4   리뷰수          182233 non-null  object 
 5   평균별점         182233 non-null  float64
 6   5점비율         182233 non-null  object 
 7   4점비율         182233 non-null  object 
 8   3점비율         182233 non-null  object 
 9   2점비율         182233 non-null  object 
 10  1점비율         182233 non-null  object 
 11  세정력/아주만족해요   182233 non-null  object 
 12  세정력/보통이에요    182233 non-null  object 
 13  세정력/다소아쉬워요   182233 non-null  object 
 14  자극도/자극없이순해요  182233 non-null  object 
 15  자극도/보통이에요    182233 non-null  object 
 16  자극도/자극이느껴져요  182233 non-null  object 
 17  거품/거품이풍성해요   182233 non-null  object 
 18  거품/보통이에요     182233 non-

In [24]:
# 한글이 한 글자도 없는 리뷰 제거
def no_korean(text):    
    pattern = rf"[^가-힣]"
    text = re.sub(pattern, ' ', text)

    return text.strip()

In [25]:
# 한글이 한 글자도 없는 리뷰 수
df["리뷰"].map(no_korean).map(lambda x: len(x)).value_counts()[0]

325

In [26]:
df[df["리뷰"].map(no_korean).map(lambda x: len(x)) == 0]["리뷰"]

447                                                                                                                                                                                                                                                                                                                                                                                                                                                     The shampoo makes my hair dryvand tangled after wash.
448                                                                                                                                                                                                                                                                                                                                                                                                                                                                It cleanses my hair well. Lots of bubbles.
846         

In [28]:
df = df[df["리뷰"].map(no_korean).map(lambda x: len(x)) != 0]
df.shape

(181908, 32)

In [29]:
182233 - 181908

325

In [30]:
df.to_csv("./data/shampoo/rawdata_final.csv", index = False)

- 데이터전처리1차와 위의 두가지 (특정 샴푸 제거, 한국어 없는 리뷰) 까지 진행한 데이터가 input 데이터

In [7]:
df = pd.read_csv("./data/shampoo/rawdata_final.csv")

C:\Users\Woojung\AppData\Local\Temp\ipykernel_14040\2830953051.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data/shampoo/rawdata_final.csv")


In [8]:
df.head()

,제품명,브랜드,원가,판매가,리뷰수,평균별점,5점비율,4점비율,3점비율,2점비율,...,향/다소아쉬워요,작성자,작성일자,리뷰,별점,사용현황,세정력,자극도,거품,향
0,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,4%,용자입니다,2025.05.19,일주일째 쓰는중인데 좋은거 같기도하고 아닌것 같기도 하고 한달뒤 후기로..ㄹ,5점만점에 5점,NaN,아주 만족해요,자극없이 순해요,거품이 풍성해요,아주 만족해요
1,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,4%,레몽몽,2025.04.28,거품이나 향은 무난한데 새치커버력이 홍보대비 좀 아쉽네요,5점만점에 4점,NaN,보통이에요,자극없이 순해요,거품이 풍성해요,아주 만족해요
2,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,4%,k202401158****,2025.05.16,얼마전에 구매했는데 쿠폰이 있어서 또 구매했어요 미리 쟁여 둘려구요,5점만점에 5점,재구매,아주 만족해요,자극없이 순해요,거품이 풍성해요,아주 만족해요
3,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,4%,k202401158****,2025.05.10,염색 하지 않고 몇년째 계속 사용하고 있어요남편 덕분에 알게 되어서 지금까지 같이 ...,5점만점에 5점,NaN,아주 만족해요,자극없이 순해요,거품이 풍성해요,아주 만족해요
4,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,4%,sweetvio****,2025.05.06,남편40대되고 흰머리가많아 이걸사줬는데그레이정도는되는것같아요,5점만점에 5점,한달이상사용,보통이에요,보통이에요,보통이에요,보통이에요


In [3]:
# jsonl 파일로 변환하여 저장
df = pd.read_csv("./data/shampoo/rawdata_final.csv")

C:\Users\Woojung\AppData\Local\Temp\ipykernel_14040\1683552531.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data/shampoo/rawdata_final.csv")


In [4]:
df.shape

(181908, 32)

In [6]:
# 파일 경로 설정
csv_file_path = './data/shampoo/rawdata_final.csv'
jsonl_file_path = './data/shampoo/rawdata_final.jsonl'

# CSV 파일 열기 및 JSONL로 저장
with open(csv_file_path, mode='r', encoding='utf-8') as csv_file, \
    open(jsonl_file_path, mode='w', encoding='utf-8') as jsonl_file:
    
    reader = csv.DictReader(csv_file)
    
    for row in reader:
        json_line = json.dumps(row, ensure_ascii=False)
        jsonl_file.write(json_line + '\n')


# 데이터 정제

- 이모지, 반복 어구 등 전처리

In [9]:
# 한글, 영어, 숫자, 공백, 직접 정한 허가 특수문자들 이외 공백처리
def clean_review(text):   
    allowed_punctuations = "!?.,%+=~&()"    
    pattern = rf"[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\s{re.escape(allowed_punctuations)}]"
    
    text = re.sub(pattern, ' ', text)

    return text.strip()

In [10]:
# 소괄호 안 부연설명 삭제
def del_bracket(text):
    # 괄호 안 부연설명 제거 (중첩 괄호 포함)
    while re.search(r'\([^()]*\)', text):
        text = re.sub(r'\([^()]*\)', ' ', text)

    # 잔여 괄호 제거
    text = text.replace("(", " ")
    text = text.replace(")", " ")
    
    return text.strip()

In [11]:
# 의미없는 반복 문자열 축약
# 반복어구
dupchars_pattern = re.compile(r'(.)\1{2,}')
dupsymbols_pattern = re.compile(r'([!?~%+=&])\1{1,}')

# 더블스페이스
doublespace_pattern = re.compile(r'\s+')

def contract_dupchars(text, n = 3): 
    if n > 0:
        text = dupchars_pattern.sub('\\1' * n, text)
        
    text = dupsymbols_pattern.sub('\\1', text)
    text = doublespace_pattern.sub(' ', text)

    return text.strip()

In [14]:
# 테스트
sample = "~~~~~⭐💚👍####…………????~~~~⭐~ -----(전처리 (개) 빡세다) ..  ...... ,,,,,,하하하하      ㅋㅋㅋㅋ~~!!zzzzㅋㅋㅋㅋㅋㅋㅋㅋ,ㅎㅎㅎㅎㅎㅎㅎㅎ졸ㄹㅕㅕㅕㅕㅕ!!!!!"
sample = clean_review(sample)
sample = del_bracket(sample)
sample = contract_dupchars(sample)

In [15]:
sample

'~ ?~ ~ .. ... ,,,하하하 ㅋㅋㅋ~!zzzㅋㅋㅋ,ㅎㅎㅎ졸ㄹㅕㅕㅕ!'

In [16]:
# 전처리 종합
def preprocessing(form):
    # 한글, 영어, 숫자, 공백, 직접 정한 허가 특수문자들 이외 공백처리
    form = clean_review(form)
    # 소괄호 안 부연설명 삭제
    form = del_bracket(form)
    # 의미없는 반복 문자열 축약
    form = contract_dupchars(form)
    
    return form

In [17]:
newdf["clean리뷰"] = newdf["리뷰"].apply(preprocessing)

C:\Users\Woojung\AppData\Local\Temp\ipykernel_30792\571489464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf["clean리뷰"] = newdf["리뷰"].apply(preprocessing)


In [18]:
newdf.head()

,제품명,브랜드,원가,판매가,리뷰수,평균별점,5점비율,4점비율,3점비율,2점비율,...,작성자,작성일자,리뷰,별점,사용현황,세정력,자극도,거품,향,clean리뷰
0,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,용자입니다,2025.05.19,일주일째 쓰는중인데 좋은거 같기도하고 아닌것 같기도 하고 한달뒤 후기로..ㄹ,5점만점에 5점,NaN,아주 만족해요,자극없이 순해요,거품이 풍성해요,아주 만족해요,일주일째 쓰는중인데 좋은거 같기도하고 아닌것 같기도 하고 한달뒤 후기로..ㄹ
1,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,레몽몽,2025.04.28,거품이나 향은 무난한데 새치커버력이 홍보대비 좀 아쉽네요,5점만점에 4점,NaN,보통이에요,자극없이 순해요,거품이 풍성해요,아주 만족해요,거품이나 향은 무난한데 새치커버력이 홍보대비 좀 아쉽네요
2,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,k202401158****,2025.05.16,얼마전에 구매했는데 쿠폰이 있어서 또 구매했어요 미리 쟁여 둘려구요,5점만점에 5점,재구매,아주 만족해요,자극없이 순해요,거품이 풍성해요,아주 만족해요,얼마전에 구매했는데 쿠폰이 있어서 또 구매했어요 미리 쟁여 둘려구요
3,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,k202401158****,2025.05.10,염색 하지 않고 몇년째 계속 사용하고 있어요남편 덕분에 알게 되어서 지금까지 같이 쓰고 있어요~,5점만점에 5점,NaN,아주 만족해요,자극없이 순해요,거품이 풍성해요,아주 만족해요,염색 하지 않고 몇년째 계속 사용하고 있어요남편 덕분에 알게 되어서 지금까지 같이 쓰고 있어요~
4,모다모다 제로그레이 블랙샴푸10 더블기획 (300g+300g),모다모다,"78,000\n원","48,280\n원",145,4.6,78%,13%,6%,1%,...,sweetvio****,2025.05.06,남편40대되고 흰머리가많아 이걸사줬는데그레이정도는되는것같아요,5점만점에 5점,한달이상사용,보통이에요,보통이에요,보통이에요,보통이에요,남편40대되고 흰머리가많아 이걸사줬는데그레이정도는되는것같아요


# 판매자 지원 리뷰

- "판매자에게 제품의 일부를 지원받았지만 저의 솔직한 리뷰입니다" 리뷰에 붙인 문구 삭제

In [19]:
# 협찬문장의 시작 패턴
start_pattern = r"(판매자(에게|로부터)|업체로부터|본 상품 후기는)"

# 협찬문장이 포함된 행 필터링
sponsored = newdf["clean리뷰"].str.contains(start_pattern)
sponsored_reviews = newdf.loc[sponsored, "clean리뷰"]

# start_pattern 으로 시작하는 문구의 위치
def find_startidx(text):
    match = re.search(start_pattern, text)
    return match.start()

start_idx = sponsored_reviews.apply(find_startidx)

# 시작 위치로부터 70자 추출
sponsored_sent = []
for idx in sponsored_reviews.index:
    pos = start_idx.loc[idx]
    sponsored_sent.append(sponsored_reviews.loc[idx][pos : pos + 70])

# 협찬문장의 전처리 (띄어쓰기보정, 문장분리)
spacing = Spacing()
kiwi = Kiwi()

cleaned_sponsored = []
for sent in sponsored_sent:
    # 띄어쓰기 제거
    sent = sent.replace(" ", "")
    # 띄어쓰기 교정
    spaced_sent = spacing(sent)
    # 문장 분리
    splited_sent = kiwi.split_into_sents(spaced_sent)
    # 첫번째 문장 (협찬문장) 추출
    cleaned_sponsored.append(splited_sent[0])

# 중복 제거한 협찬 문장
sponsored_patterns = list({sent[0] for sent in cleaned_sponsored})

C:\Users\Woojung\AppData\Local\Temp\ipykernel_30792\1016952794.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sponsored = newdf["clean리뷰"].str.contains(start_pattern)


In [21]:
pattern1 = []
pattern2 = []
pattern3 = []
pattern4 = []
for sent in sponsored_patterns:
    if "판매자에게" in sent:
        pattern1.append(sent)
    elif "판매자로부터" in sent:
        pattern2.append(sent)
    elif "업체로부터" in sent:
        pattern3.append(sent)
    elif "본 상품 후기는" in sent:
        pattern4.append(sent)

In [23]:
# "판매자에게" 로 시작
pattern1

['판매자에게 제품 비 일부를 받았지만 솔직한 저의 후기입니다',
 '판매자에게 제품비를 일부 지원받았지만 저의 솔직한 후기입니다.',
 '판매자에게 이벤트 상금을 지원받아 직접 구매 후 솔직한 의견으로 작성하였습니다청 미정 다시마 에센스가 유명해서 잘 쓰고 있었는데',
 '판매자에게 제품의 일부를 지원받았지만 저의 솔직한 후기입니다.',
 '판매자에게 제품비의 일부를 지원받았지만 저의 솔직한 후기입니다 써보고 맘에 들어 꾸준히 쓰고 있는 제품입니다',
 '판매자에게 제품비의 일부를 지원받았지만 저의 솔직한 후기입니다',
 '판매자에게 제품비의 일부를 지원받았지만 저의 솔직한 후기입니다 점점 나이를 먹으면서 머리카락 관리 안 할 수가 없더라고요.',
 '판매자에게 제품비의 일부를 받았지만 저의 솔직한 후기입니다.',
 '판매자에게 제품비를 일부 지원받았지만 저의 솔직한 후기입니다!',
 '판매자에게 제품비의 일부를 지원받았지만 저의 솔직한 후기입니다 머리숱도 없는데 오후만 되면 머리가 떡져서 스트레스를 많이 받았는',
 '판매자에게 제품비의 일부를 지원받았지만, 저의 가족의 솔직한 후 기랍니다.!',
 '판매자에게 제품비의 일부를 지원받았으나 저의 솔직한 후기를 작성하였습니다.',
 '판매자에게 제품 비용 일부를 지원받았지만 사용 후에 주관적으로 솔직하게 작성한 후기입니다.',
 '판매자에게 제품비의 일부를 지원받았지만 저의 솔직한 사용 후기입니다.',
 '판매자에게 제품 비 일부를 지원받았지만 저의 솔직한 후기입니다 샴푸는 계속해서 닥터그루트로 온 가족이 사용하고 있어요.',
 '판매자에게 제품 비 일부를 지원받았지만 사용 후 저의 솔직한 후기로 리뷰적었습니다',
 '판매자에게 제품 지원을 받았지만 사용해보고 직접 작성하는 주관적인 후기입니다.',
 '판매자에게 제품비의 일부를 지원받았지만 저의 솔직한 후기입니다가려운 피부용인데 진짜 덜 가려운 것 같아서 두통째 구매했어요!',
 '판매자에게 이벤트 상금을 지원받았지만 직접 구매 후 솔직한 의견으로 작성하였습

In [24]:
# "판매자로부터" 로 시작
pattern2

['판매자로부터 제품 비 일부를 지원받았지만 솔직하게 작성한 후기입니다',
 '판매자로부터 제품비의 일부를 지원받았지만 저의 솔직한 후기입니다',
 '판매자로부터 제품의 일부 비용을 지원받았습니다만 제 솔직한 후기를 올립니다.',
 '판매자로부터 제품 비 일부를 지원받았지만 저의 솔직한 리뷰입니다']

In [25]:
# "업체로부터" 로 시작
pattern3

['업체로부터 제품을 지원받아 사용 후 솔직하게 작성한 후기입니다',
 '업체로부터 제품을 할인받아 구매해서 솔직하게 작성한 리뷰입니다',
 '업체로부터 제품을 제공받아 실사용 후 솔직하게 작성된 리뷰입니다',
 '업체로부터 지원받아 사용 후 작성하는 리뷰입니다']

In [26]:
# "본 상품 후기는" 로 시작
pattern4

['본 상품 후기는 체험단으로 선정되어 CJ올리브영으로부터 위상품을 무료로 제공받아 작성한 것입니다.',
 '본 상품 후기는 체험담으로 선정되어 CJ올리브영으로부터 위상품으로 무료로 제공받아 작성한 것입니다.']

In [35]:
def del_sponsored(text):
    sponsored_pattern = r'''
        (판매자(에게|로부터)|업체로부터|본\s상품\s후기는).{0,70}?
        (후기(입니다|입니다타|에요|입니\s?다)?|
         리뷰(입니다|했습니다|적었습니다|하였습니다|입니다요)?|
         작성하였습니다|기남겨요|기랍니다|흐기입니다|전달합니다|올립니다|것\s?입니다)
        [!.~\s]{0,2}
    '''
    sponsored_pattern = re.compile(sponsored_pattern, flags=re.VERBOSE)
    text = sponsored_pattern.sub(' ', text)
    
    return text.strip()

In [36]:
# 테스트
del_sponsored("향은 멘톨이 들어가서 그런가 시원한향이나고쿨링감이 시원하게 좋은느낌이 오래가고 다른샴푸는 양모효과라고 하면서 머리가 뻣뻣해지는 느낌이 나는데 이샴푸는 머리결이 더 부드러워지고 머리가 조금 덜 간지러워서 좋은것같아요 다쓰고나면 재구매 의향이 있습니다 본 상품 후기는 체험단으로 선정되어 CJ올리브영으로부터 위 상품을 무료로 제공받아 작성한 것입니다.")

'향은 멘톨이 들어가서 그런가 시원한향이나고쿨링감이 시원하게 좋은느낌이 오래가고 다른샴푸는 양모효과라고 하면서 머리가 뻣뻣해지는 느낌이 나는데 이샴푸는 머리결이 더 부드러워지고 머리가 조금 덜 간지러워서 좋은것같아요 다쓰고나면 재구매 의향이 있습니다'

In [37]:
newdf["clean리뷰"] = newdf["clean리뷰"].apply(del_sponsored)

C:\Users\Woojung\AppData\Local\Temp\ipykernel_30792\1255005365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf["clean리뷰"] = newdf["clean리뷰"].apply(del_sponsored)


In [38]:
# clean리뷰의 길이 통계값
lengths = np.array([len(x) for x in newdf["clean리뷰"]])
print(np.mean(lengths), np.median(lengths), np.min(lengths), np.max(lengths))
print(np.percentile(lengths, 25), np.percentile(lengths, 75))

129.35936411078126 74.0 0 1331
41.0 161.0


In [39]:
newdf = newdf[newdf["clean리뷰"].map(lambda x: len(x) >= 10)]

- 리뷰 길이가 10 미만인 51개 행 삭제

In [40]:
newdf.shape

(182182, 33)

In [43]:
newdf.to_csv("./data/shampoo/oliveyoung_cleaned2.csv", index = False)